In [18]:
import tensorflow as tf
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt

from sklearn.metrics import f1_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score, accuracy_score, plot_confusion_matrix

In [4]:
# Read & split train test
filename = './data/clean_entire_corpus.csv'
df = pd.read_csv(filename)
df_train = df[df['source'] == 'train'].copy()
df_test = df[df['source'] == 'test'].copy()

In [5]:
# Encoding
def get_vocabulary(iter_sents):
    vocab = set()
    max_length = 0
    for text in iter_sents:
        tokens = text.split(' ')
        if len(tokens) > max_length:
            max_length = len(tokens)
        vocab.update(tokens)
    return vocab, max_length

vocab, max_length = get_vocabulary(df_train['clean_text'])

tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(df_train['clean_text'])
sequences_train = tokenizer.texts_to_sequences(df_train['clean_text'])
sequences_test = tokenizer.texts_to_sequences(df_test['clean_text'])

In [6]:
# Padding
padded_seqs_train = tf.keras.preprocessing.sequence.pad_sequences(sequences_train, maxlen=max_length,
                                                            padding='post', truncating='post')
padded_seqs_test = tf.keras.preprocessing.sequence.pad_sequences(sequences_test, maxlen=max_length,
                                                            padding='post', truncating='post')

In [7]:
# Data sets
X_train = padded_seqs_train
X_test = padded_seqs_test
y_train = df_train['target'].values
y_test = df_test['target'].values

train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(1000).batch(100)
test_ds = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(100)

In [8]:
# Build an index for embeddings
embeddings_index = {}
EMBEDDINGS_PATH = '/datascience/embeddings'
EMBEDDINGS_LENGTH = 50
print('Reading lines')
with open(os.path.join(EMBEDDINGS_PATH, 'glove.6B.50d.txt'), encoding='utf-8') as f:
    lines = f.readlines()

print('Extract embeddings')
for line in lines:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs

print('Create embeddings matrix')
word_index = tokenizer.word_index
embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDINGS_LENGTH))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector


Reading lines
Extract embeddings
Create embeddings matrix


In [9]:
# Model
model = tf.keras.Sequential()
model.add(a := tf.keras.layers.Embedding(input_dim=len(word_index)+1, output_dim=EMBEDDINGS_LENGTH, input_length=max_length,
                                         weights=[embedding_matrix], trainable=False))
model.add(b := tf.keras.layers.Flatten())
model.add(c := tf.keras.layers.Dense(256, activation='relu'))
model.add(d := tf.keras.layers.Dense(1, activation='sigmoid'))

optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['acc'])

model.fit(train_ds, epochs=10, validation_data=test_ds)

Epoch 1/10
77/77 [==============================] - 1s 10ms/step - loss: 0.4963 - acc: 0.7688 - val_loss: 0.4694 - val_acc: 0.7898
Epoch 2/10
77/77 [==============================] - 1s 7ms/step - loss: 0.3767 - acc: 0.8362 - val_loss: 0.4643 - val_acc: 0.7882
Epoch 3/10
77/77 [==============================] - 1s 7ms/step - loss: 0.3074 - acc: 0.8778 - val_loss: 0.4740 - val_acc: 0.7824
Epoch 4/10
77/77 [==============================] - 1s 7ms/step - loss: 0.2430 - acc: 0.9117 - val_loss: 0.5003 - val_acc: 0.7858
Epoch 5/10
77/77 [==============================] - 1s 7ms/step - loss: 0.1884 - acc: 0.9380 - val_loss: 0.5214 - val_acc: 0.7818
Epoch 6/10
77/77 [==============================] - 1s 7ms/step - loss: 0.1489 - acc: 0.9544 - val_loss: 0.5481 - val_acc: 0.7809
Epoch 7/10
77/77 [==============================] - 1s 7ms/step - loss: 0.1250 - acc: 0.9652 - val_loss: 0.5740 - val_acc: 0.7769
Epoch 8/10
77/77 [==============================] - 1s 7ms/step - loss: 0.1057 - acc: 0.9

In [19]:
def show_performance(model):
    model.fit(X_train, y_train)
    pred = model.predict(X_test).reshape(1, -1)[0]
    pred = np.around(pred).astype(int)
    print(pred)

    print(f"Accuracy : {accuracy_score(y_test, pred)}")
    print(f"F1-score : {f1_score(y_test, pred)}")

    disp = plot_confusion_matrix(model, X_test, y_test,
                        display_labels=['0', '1'],
                        cmap=plt.cm.Blues,
                        normalize='pred')
    disp.ax_.grid(False)
    
show_performance(model)

238/238 [==============================] - 1s 4ms/step - loss: 0.0739 - acc: 0.9773
[1 1 1 ... 0 1 1]
Accuracy : 0.7876187557462457
F1-score : 0.7375993941688753


ValueError: plot_confusion_matrix only supports classifiers